In [1]:
"""
Indentifying images with and without animals via classification.Images with animals
are labeled and "positive", images without are labeled "negative".

Based on: 
    - https://medium.com/@1297rohit/transfer-learning-from-scratch-using-keras-339834b153b9
    - Uses Transfer Learning
    - Based on a VGG (pre trained) model
    
@Author: D. Barros
@Version: 1.0
"""

import os

path = "C:/Users/DBarros/Documents/SAFArI/Machine Learning tryouts/Datasets/Biota01 - positivo"
#path = "M:\Datasets\Biota01 - positivos"

os.chdir(path)
print(os.getcwd())

C:\Users\DBarros\Documents\SAFArI\Machine Learning tryouts\Datasets\Biota01 - positivo


In [2]:
import keras
from keras.models import Model
from keras.layers import Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [7]:
batch_size = 32
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory="train", batch_size=batch_size, target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory="test", batch_size=batch_size, target_size=(224,224))


Found 4192 images belonging to 2 classes.
Found 480 images belonging to 2 classes.


In [8]:
from keras.applications.vgg16 import VGG16
vggmodel = VGG16(weights='imagenet', include_top=True)

In [9]:
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
#Setting the 19 layers in the model to be static as I do not want them to be trained again. This will be changed.
for layers in (vggmodel.layers)[:19]:
    print(layers)
    layers.trainable = False

In [11]:
#Adjusting the pre trained VGG model. Pre trained model is trained for 1000 classes. I only want 2 classes.
X= vggmodel.layers[-2].output
predictions = Dense(2, activation="softmax")(X)
model_final = Model(inputs = vggmodel.input, outputs = predictions)

In [12]:
#Compiling the new model
model_final.compile(loss = "categorical_crossentropy",
                    optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

In [13]:
model_final.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [14]:
#Early stoping allows for the model to stop if accuracy hasn't improved in "patience" epochs.
#Model chekpoint allows for saving of the best model

from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

model_final.fit_generator(generator= traindata,
                          steps_per_epoch= traindata.samples/batch_size,
                          epochs= 5,
                          validation_data= testdata,
                          validation_steps=testdata.samples/batch_size,
                          callbacks=[checkpoint,early])

model_final.save_weights("vgg16_1.h5")



Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
7717/7716 [==============================] - ETA: -5s - loss: 0.1982 - accuracy: 0.9122
Epoch 00001: val_accuracy improved from -inf to 0.83499, saving model to vgg16_1.h5
7717/7716 [==============================] - 46166s 6s/step - loss: 0.1982 - accuracy: 0.9122 - val_loss: 0.3712 - val_accuracy: 0.8350
Epoch 2/5
7717/7716 [==============================] - ETA: -5s - loss: 0.1456 - accuracy: 0.9365
Epoch 00002: val_accuracy improved from 0.83499 to 0.88723, saving model to vgg16_1.h5
7717/7716 [==============================] - 46245s 6s/step - loss: 0.1456 - accuracy: 0.9365 - val_loss: 0.2587 - val_accuracy: 0.8872
Epoch 3/5
7717/7716 [==============================] - ETA: -5s - loss: 0.1271 - accuracy: 0.9459
Epoch 00003: val_accuracy did not improve from 0.88723
7717/7716 [==============================] - 46496s 6s/step - loss: 0.1271 - accuracy: 0.9459 - val_loss: 0.3004 - val_accuracy: 0.8

In [3]:
# Loading the model which will predict our results
model_final = keras.models.load_model('./vgg16_1.h5')

In [4]:
# load and iterate predictions
predict_data = ImageDataGenerator()
predictdata = predict_data.flow_from_directory(directory="predict", target_size=(224,224))

Found 78 images belonging to 1 classes.


In [5]:
#Make some predictions
pred=model_final.predict(predictdata)
print("done")

done


In [8]:
import numpy as np
import pandas as pd
predicted_class_indices=np.argmax(pred,axis=1)

labels = (traindata.class_indices)
labels = dict((v,k) for k,v in labels.items())
preds= [labels[k] for k in predicted_class_indices]
probs = [float(max(i)) for i in pred]

filenames=predictdata.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":preds,
                      "Probability":probs,
                      })

In [9]:
pd.options.display.max_rows = 99
results

,Filename,Predictions,Probability
0,predictions\11300077.JPG,positivos,0.999677
1,predictions\11300078.JPG,negativos,0.999466
2,predictions\11300079.JPG,positivos,0.999373
3,predictions\11300080.JPG,positivos,0.999940
4,predictions\11300081.JPG,positivos,1.000000
5,predictions\11300082.JPG,positivos,0.999998
6,predictions\11300083.JPG,positivos,0.999903
7,predictions\11300084.JPG,negativos,0.988383
8,predictions\11300085.JPG,positivos,0.999994
9,predictions\11300086.JPG,positivos,0.979048


In [10]:
#Saving the results in a csv file
results.to_csv("results.csv",index=False)

In [12]:
#Saving the results in a jason file
import json
dResults = {}
dResults['results'] = []
for i in range(len(results.Filename)):
    dResults["results"].append({
        "Filename" : results.Filename[i],
        "Predictions" : results.Predictions[i],
        "Probability" : results.Probability[i],
    })

In [15]:
with open('data.JSON', 'w') as outfile:
    json.dump(dResults, outfile)